Homework1 - https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [1]:
#all imports and constants
import requests
import pandas

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error


jan2023_data_set_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
feb2023_data_set_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

/Users/madhukarbs/src/jupyter_notebooks/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 3.3.6'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
data_frame_jan = pandas.read_parquet(jan2023_data_set_url)
data_frame_feb = pandas.read_parquet(feb2023_data_set_url)


In [3]:
print("No. of columns: {}".format(len(data_frame_jan.columns)))


No. of columns: 19


In [4]:
data_frame_jan.columns
data_frame_jan["duration"] = data_frame_jan["tpep_dropoff_datetime"] - data_frame_jan["tpep_pickup_datetime"]
data_frame_jan.duration = data_frame_jan.duration.apply((lambda td: td.total_seconds() / 60))
data_frame_feb.columns
data_frame_feb["duration"] = data_frame_feb["tpep_dropoff_datetime"] - data_frame_feb["tpep_pickup_datetime"]
data_frame_feb.duration = data_frame_feb.duration.apply((lambda td: td.total_seconds() / 60))

sd = data_frame_jan.duration.std()
print(sd)


42.59435124195458


In [5]:
og_length = data_frame_jan.size

In [6]:
# count=0
# outliers=0
# for r in data_frame_jan.duration:
#     if r>=1 and r<=60:
#         count+=1
#     else:
#         outliers+=1

# print(count)
# print(outliers)

data_frame_jan = data_frame_jan[(data_frame_jan.duration >= 1) & (data_frame_jan.duration <= 60)]
data_frame_feb = data_frame_feb[(data_frame_feb.duration >= 1) & (data_frame_feb.duration <= 60)]

q3_answer = data_frame_jan.size*100/og_length
print("{}%".format(q3_answer))

98.1220282212598%


In [7]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

In [8]:
data_frame_jan[categorical] = data_frame_jan[categorical].astype(str)
data_frame_feb[categorical] = data_frame_feb[categorical].astype(str)

In [9]:
train_dicts = data_frame_jan[categorical + numerical].to_dict(orient='records')
train_dicts_feb = data_frame_feb[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)


In [11]:
target = 'duration'
y_train = data_frame_jan[target].values
y_val = data_frame_jan[target].values
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_val, y_pred)


7.658406582175197

In [12]:
# dv_feb = DictVectorizer()
X_feb = dv.transform(train_dicts_feb)


In [14]:
y_val_feb = data_frame_feb[target].values
y_pred_feb = lr.predict(X_feb)

root_mean_squared_error(y_val_feb, y_pred_feb)


7.820096870991671